In [1]:
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import glob
import shutil
import sys
import torch
from PIL import Image

In [2]:
!git clone https://github.com/ultralytics/yolov5.git

Cloning into 'yolov5'...
remote: Enumerating objects: 12727, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 12727 (delta 45), reused 48 (delta 22), pack-reused 12633
Receiving objects: 100% (12727/12727), 12.95 MiB | 27.69 MiB/s, done.
Resolving deltas: 100% (8754/8754), done.


In [ ]:
ROOT_DIR  = '/kaggle/input/tensorflow-great-barrier-reef/'
# CKPT_DIR  = '/kaggle/input/greatbarrierreef-yolov5-train-ds'
CKPT_PATH = '/content/drive/MyDrive/hackathon2/web/best.pt' # by @steamedsheep
IMG_SIZE  = 1000
CONF      = 0.15
IOU       = 0.5
AUGMENT   = True